<a href="https://colab.research.google.com/github/ucfilho/optim_raianars/blob/master/ABCOptmi_from_C_language_jul_15_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from random import randint

In [0]:
def CalculateFitness(fun):
    result=0
    if(fun>=0):
      result=1/(fun+1)
    else:
      result=1+fabs(fun)
    return result

In [0]:
def function( sol):
  #Schwefel(x):
  x=sol  
  summ=0
  for i in range(len(x)):
    new=x[i]*np.sin((abs(x[i]))**0.5)
    summ=summ+new
    top=(418.9829*len(x)-summ) 
  
  return top;


In [0]:
def MemBestSource(GlobMin,GlobPars,Foods):
  FoodNumber=len(Foods[:,0])
  D=len(Foods[0,:])
  solution=np.zeros(D)
  for i in range(FoodNumber):
    if (f[i]<GlobMin):
      GlobMin=f[i]
      for j in range(D):
        GlobPars[j]=Foods[i,j]
  return GlobMin,GlobPars,Foods

In [0]:
def init(index):
  global Foods
  global trial
  global f
  global fitness
  global lb
  global ub
  
  D=len(Foods[0,:])
  FoodNumber=len(Foods[:,0])
  solution=np.zeros(D)
  
  for j in range(D):
    r=np.random.random()
    Foods[index,j]=r*(ub-lb)+lb
    solution[j]=Foods[index,j]
    
  f[index]=function(solution)
  fitness[index]=CalculateFitness(f[index])
  trial[index]=0

In [0]:
def initial(f,Foods,GlobMin,GlobPars):
  D=len(Foods[0,:])
  FoodNumber=len(Foods[:,0])
  for i in range(FoodNumber):
    init(i) 
  GlobMin=f[0]
  for i in range(D):
    GlobPars[i]=Foods[0,i]
  return  f,Foods,GlobMin,GlobPars


In [0]:
def EmployedBees(trial,Foods):
  FoodNumber=len(Foods[:,0])
  D=len(Foods[0,:])
  solution=np.zeros(D)
  for i in range(FoodNumber):
    r = np.random.random()
    par2chan= int(r*D)
    r = np.random.random()
    neighbour=int(r*D)
    if(neighbour >= NP):
      neighbour=NP-1
      
    while(neighbour==i):
      r = np.random.random()
      neighbour=int(r*FoodNumber)
      if(neighbour >= NP):
        neighbour=NP-1
    for j in range(D):
      solution[j]=Foods[i,j]
#/*v_{ij}=x_{ij}+\phi_{ij}*(x_{kj}-x_{ij}) */      
    r = np.random.random()
    A=Foods[i,par2chan]
    B=Foods[neighbour,par2chan]
    solution[par2chan]=Foods[i,par2chan]+(A-B)*(r-0.5)*2;
    if (solution[par2chan]<lb):
      solution[par2chan]=lb
    if (solution[par2chan]>ub):
      solution[par2chan]=ub
    ObjValSol=function(solution)
    FitnessSol=CalculateFitness(ObjValSol)
    if(FitnessSol>fitness[i]):
      trial[i]=0
      for j in range(D):
        Foods[i,j]=solution[j]
      f[i]=ObjValSol
      fitness[i]=FitnessSol
    else:
      trial[i]=trial[i]+1 
      
  return trial,Foods

In [0]:
def CalcProb(fitness,prob):
  FoodNumber=len(fitness)
  maxfit=fitness[0]
  for i in range(1,FoodNumber):    
    if (fitness[i]>maxfit):
      maxfit=fitness[i]
  for i in range(FoodNumber):
    prob[i]=(0.9*(fitness[i]/maxfit))+0.1
  return fitness,prob


In [0]:
def OnlookerBees(trial,Foods):
  D=len(Foods[0,:])
  solution=np.zeros(D)
  FoodNumber=len(Foods[:,0])
  i=0
  t=0
  while(t<FoodNumber):
    r = np.random.random()
    if(r<prob[i]): 
      r = np.random.random()
      t=t+1
      par2chan=int(r*D)
      r = np.random.random()
      neighbour=int(r*FoodNumber)

      while(neighbour==i):
        r = np.random.random()
        neighbour=int(r*FoodNumber)
      for j in range(D):
        solution[j]=Foods[i,j]

      r = np.random.random()
      A=Foods[i,par2chan]
      B=Foods[neighbour,par2chan]
      solution[par2chan]=A+(A-B)*(r-0.5)*2

      if (solution[par2chan]<lb):
              solution[par2chan]=lb
      if (solution[par2chan]>ub):
              solution[par2chan]=ub
      ObjValSol=function(solution)
      FitnessSol=CalculateFitness(ObjValSol)
              
      if(FitnessSol>fitness[i]):
        trial[i]=0
        for j in range(D):
          Foods[i,j]=solution[j]
        f[i]=ObjValSol
        fitness[i]=FitnessSol
      else:
        trial[i]=trial[i]+1
      i=i+1
      if (i==FoodNumber):
        i=0
  return trial,Foods


In [0]:
def ScoutBees(Foods,trial):
  FoodNumber=len(Foods[:,0])
  maxtrialindex=0
  for i in range(1,FoodNumber):
    if (trial[i]>trial[maxtrialindex]):
      maxtrialindex=i
  if(trial[maxtrialindex]>=limit):
    init(maxtrialindex)
  return trial,Foods


In [11]:
'''/*Main program of the ABC algorithm*/'''
NP=175
limit=100
D=3
lb=-500
ub=500
runtime=4
maxCycle=300
FoodNumber=int(NP)
f=np.zeros(FoodNumber) 
Foods=np.zeros((FoodNumber,D)) 
solution=np.zeros(D)
fitness=np.zeros(FoodNumber)
trial=np.zeros(FoodNumber)
prob=np.zeros(FoodNumber)
GlobPars=np.zeros(D)
GlobMins=np.zeros(runtime)
GlobMin=1e99

for run in range(runtime):
  f,Foods,GlobMin,GlobPars= initial(f,Foods,GlobMin,GlobPars)
  GlobMin,GlobPars,Foods=MemBestSource(GlobMin,GlobPars,Foods)
  mean=0
  for iter in range(maxCycle):
    trial,Foods=EmployedBees(trial,Foods)
    fitness,prob=CalcProb(fitness,prob)
    trial,Foods=OnlookerBees(trial,Foods)
    GlobMin,GlobPars,Foods=MemBestSource(GlobMin,GlobPars,Foods)
    trial,Foods=ScoutBees(Foods,trial)

  for j in range(D):
    print("GlobalParam[",j+1,"]:", GlobPars[j])
  print("run",run+1,"GBest=",GlobMin," \n");
  GlobMins[run]=GlobMin
  mean=mean+GlobMin
mean=mean/runtime;
print("Means of",runtime,"runs:",mean,"\n")

GlobalParam[ 1 ]: 420.9687463754004
GlobalParam[ 2 ]: 420.96874604389336
GlobalParam[ 3 ]: 420.96874670138095
run 1 GBest= 3.818269851763034e-05  

GlobalParam[ 1 ]: 420.9687462912745
GlobalParam[ 2 ]: 420.96874657521005
GlobalParam[ 3 ]: 420.96874625892974
run 2 GBest= 3.818269851763034e-05  

GlobalParam[ 1 ]: 420.9687462120315
GlobalParam[ 2 ]: 420.96874613600477
GlobalParam[ 3 ]: 420.9687460529805
run 3 GBest= 3.818269851763034e-05  

GlobalParam[ 1 ]: 420.9687466625984
GlobalParam[ 2 ]: 420.9687465500514
GlobalParam[ 3 ]: 420.96874624640674
run 4 GBest= 3.818269851763034e-05  

Means of 4 runs: 9.545674629407586e-06 



In [12]:
print(np.median(GlobMins))

3.818269851763034e-05


In [13]:
3.818269851763034e-05/4

9.545674629407586e-06